# HTML to SQL

In this tutorial, we'll take the result of scraping a web page and make a SQL database from it. Over the course of the tutorial, I'll show you how to scrape a page, make Python data structure from it, devise a SQL schema for the data, and then use Python (with pg8000) to insert the data into a PostgreSQL database.

## Introducing Menupages

As a target for this exercise, we'll pick some HTML that presents a moderate challenge: [the Morningside Heights listing from Menupages](http://www.menupages.com/restaurants/all-areas/morningside-heights/all-cuisines/). (Menupages is now owned by and largely superceded by Seamless, but in my formative period as a New Yorker in the middle of the first decade of the 2000s it was *the place* to go for information about where to get food in Manhattan. Menupages doesn't make its data available in a non-HTML format, so if we want to get our hands on the data we need to scrape it from the website.

To make things easier on the Menupages server, and so that this tutorial doesn't break when/if Menupages changes their layout, I've made a copy of the Morningside Heights directory HTML and uploaded it to Github [here](https://raw.githubusercontent.com/ledeprogram/data-and-databases/master/menupages-morningside-heights.html). We'll download and use this file locally instead of fetching it from Menupages directly.

> Note: If you're trying to reproduce my results from the Menupages site itself, note that I slightly modified the source code to correct some bad HTML in the `<thead>` tag, which was preventing `html.parser` from parsing the page into a tree properly.

## Making the data structure

The data structure that we'll want to end up with as a sort of "intermediary" between the HTML and SQL is a list of dictionaries, with each dictionary having keys and values corresponding to the data for each restauraunt. Eyeballing the page, I'd say that the data structure we'd want to end up with might look like this:

    [
        {'name': '176 Presbyterian Deli',
         'cuisines': [],
         'price': None},
        {'name': 'Amir's Grill',
         'cuisines': ['Middle Eastern'],
         'price': None}
        {'name': 'Artopolis Espresso Cafe',
         'cuisines': ['Coffee & Tea', 'Sandwiches', 'Crepes'],
         'price': '$'}
        },
        ... more of these probably ...
    ]
    
A data structure like this clearly separates out all of the individual pieces of data available to us in the HTML, and allows us to easily transform the data into a different format (e.g., CSV, SQL, etc.).

> Note: This isn't a "standard" format or anything—it's just something I made up, based on what is present in the HTML. I might decide later to revise what the data structure looks like, if I find out something about the data that wasn't immediately apparent from just looking at the rendered HTML. It's worth noting that there's no automated tool to "force" the HTML into this data format. (In fact, that's the whole point of this tutorial.)

## Scraping the HTML

By prodding at the source code with Developer Tools, I isolated the important part of the HTML to this little bit here:

![screenshot of dev tools on menupages](http://static.decontextualize.com/snaps/menupages-devtools.png)

At first glance: There's a table with class `search-results` that has a `<tbody>` tag that is the parent tag for a series of `<tr>` tags that represent each restaurant, row-by-row. Inside each `<tr>` tag, there are a number of `<td>`s that look useful: one with class `name-address` that appears to have the name and address of the restaurant; another with class `price` that has the price of the restaurant. The "cuisines" of the restaurant can be obtained either from the `name-address` `<td>`, or from a mysterious `<td>` without a class that appears to be in the table but not displayed anywhere. (That's weird, but not unheard of: it's likely hidden by a CSS class, and the data is probably present only to be used by some Javascript on the page.)

As a first attempt, let's just try to print the *name* of every restaurant. First, we'll load the data and create a Beautiful Soup object.

In [40]:
from bs4 import BeautifulSoup

raw_html = open("menupages-morningside-heights.html").read()
soup = BeautifulSoup(raw_html, "html.parser")

Now, we'll find that `<table>` tag and iterate through its child `<tr>`s:

In [41]:
search_table = soup.find('table', {'class': 'search-results'}) # the search-results table
table_body = search_table.find('tbody') # get only the table body
for tr_tag in table_body.find_all('tr'):
    name_address_tag = tr_tag.find('td', {'class': 'name-address'})
    a_tag = name_address_tag.find('a')
    print(a_tag.string)

Milano Market
Massawa
China Place
Subsconscious
Famous Famiglia
Kitchenette
V & T Pizza
New Aroma
Peking Garden
Tom's Restaurant
Pisticci
Deluxe
Toast
Tom's Delicious Pizza
West Place
Che' Bella Pizza
Ajanta
Panino Sportivo Roma
Max Soha
Strokos Pizza
Camille's
Amsterdam Restaurant
Nussbaum & Wu
Amir's Grill
M2M - Morning To Midnight
The Mill
Le Monde
Melba's
Chuck E Cheese's
Haagen-Dazs
Oren's
Dinosaur Bar-B-Que
Symposium Greek Restaurant
Koronet Pizza
The Heights Bar & Grill
Cafe Nana
Hamilton Deli
Community Food & Juice
Haakon's Hall
El Porton
Brad's
Mel's Burger Bar
Bettolona
Five Guys
Nikko
Falafel on Broadway
Sushi Sushi
Insomnia Cookies
Cafe Amrita
Pinkberry
Artopolis Espresso Cafe
Max Caffe
Chipotle
Chokolat Patisserie
Chokolat Patisserie
Joe's G-H Deli
Joe the Art of Coffee
Levain Bakery
Silvana
Bier International
Vegenation
Flat Top
Kuro Kuma
Vinateria
Henan Cart
Vine
El Paso Truck
Mama's Fried Chicken and Pizza
Chapati House
Dig Inn Seasonal Market
Uncle Luoyang
Pita Grill
K

Okay, that looks okay so far! Let's continue the work. The next easiest-hanging fruit is the price. Looking through the HTML, we can see that the price is recorded in one of two ways: either as a `N/A` string inside the `price`-classed `<td>` tag, or as a string inside of a `span` that is the direct descendent of that tag. We'll check for both.

In [42]:
search_table = soup.find('table', {'class': 'search-results'}) # the search-results table
table_body = search_table.find('tbody') # get only the table body
for tr_tag in table_body.find_all('tr'):
    name_address_tag = tr_tag.find('td', {'class': 'name-address'})
    a_tag = name_address_tag.find('a')
    name = a_tag.string
    price_tag = tr_tag.find('td', {'class': 'price'})
    price_span_tag = price_tag.find('span')
    if price_span_tag:
        price = price_span_tag.string
    else:
        price = None
    print(name, "-", price)

Milano Market - 2
Massawa - None
China Place - None
Subsconscious - None
Famous Famiglia - None
Kitchenette - None
V & T Pizza - None
New Aroma - 1
Peking Garden - 1
Tom's Restaurant - 1
Pisticci - 1
Deluxe - 1
Toast - 2
Tom's Delicious Pizza - 2
West Place - 1
Che' Bella Pizza - None
Ajanta - None
Panino Sportivo Roma - 2
Max Soha - 2
Strokos Pizza - 3
Camille's - 1
Amsterdam Restaurant - 1
Nussbaum & Wu - None
Amir's Grill - 1
M2M - Morning To Midnight - 1
The Mill - 2
Le Monde - 1
Melba's - 1
Chuck E Cheese's - None
Haagen-Dazs - 1
Oren's - None
Dinosaur Bar-B-Que - 3
Symposium Greek Restaurant - None
Koronet Pizza - 1
The Heights Bar & Grill - 1
Cafe Nana - None
Hamilton Deli - None
Community Food & Juice - None
Haakon's Hall - None
El Porton - None
Brad's - 1
Mel's Burger Bar - 2
Bettolona - 1
Five Guys - 1
Nikko - 2
Falafel on Broadway - 1
Sushi Sushi - 1
Insomnia Cookies - 1
Cafe Amrita - 1
Pinkberry - 2
Artopolis Espresso Cafe - 1
Max Caffe - 2
Chipotle - 1
Chokolat Patisserie 

At this point, the code is getting a little bit ugly, so I'm going to try to compartmentalize it a bit by moving some of the code for extracting information from the `<tr>` tags into separate functions.

In [92]:
def get_name(tr_tag):
    name_address_tag = tr_tag.find('td', {'class': 'name-address'})
    a_tag = name_address_tag.find('a')
    return str(a_tag.string) # call str() to convert to regular string, not bs4 navigable string type
    
def get_price(tr_tag):
    price_tag = tr_tag.find('td', {'class': 'price'})
    price_span_tag = price_tag.find('span')
    if price_span_tag:
        return int(price_span_tag.string)
    else:
        return 0

Now we can rewrite our loop, but in a much cleaner way:

In [93]:
search_table = soup.find('table', {'class': 'search-results'}) # the search-results table
table_body = search_table.find('tbody') # get only the table body
for tr_tag in table_body.find_all('tr'):
    name = get_name(tr_tag)
    price = get_price(tr_tag)
    print(name, "-", price)

Milano Market - 2
Massawa - 0
China Place - 0
Subsconscious - 0
Famous Famiglia - 0
Kitchenette - 0
V & T Pizza - 0
New Aroma - 1
Peking Garden - 1
Tom's Restaurant - 1
Pisticci - 1
Deluxe - 1
Toast - 2
Tom's Delicious Pizza - 2
West Place - 1
Che' Bella Pizza - 0
Ajanta - 0
Panino Sportivo Roma - 2
Max Soha - 2
Strokos Pizza - 3
Camille's - 1
Amsterdam Restaurant - 1
Nussbaum & Wu - 0
Amir's Grill - 1
M2M - Morning To Midnight - 1
The Mill - 2
Le Monde - 1
Melba's - 1
Chuck E Cheese's - 0
Haagen-Dazs - 1
Oren's - 0
Dinosaur Bar-B-Que - 3
Symposium Greek Restaurant - 0
Koronet Pizza - 1
The Heights Bar & Grill - 1
Cafe Nana - 0
Hamilton Deli - 0
Community Food & Juice - 0
Haakon's Hall - 0
El Porton - 0
Brad's - 1
Mel's Burger Bar - 2
Bettolona - 1
Five Guys - 1
Nikko - 2
Falafel on Broadway - 1
Sushi Sushi - 1
Insomnia Cookies - 1
Cafe Amrita - 1
Pinkberry - 2
Artopolis Espresso Cafe - 1
Max Caffe - 2
Chipotle - 1
Chokolat Patisserie - 5
Chokolat Patisserie - 1
Joe's G-H Deli - 2
Joe 

Getting the "cuisines" is a bit more difficult. I think we'll try to use the mysterious `<td>` tag that doesn't have a class. In order to use this, we have to do something we haven't done yet, which is grab a matching element *by position* instead of by class. This function takes the `<tr>` tag object and gets the content of the fifth (index 4) `<td>` tag, splitting it on a comma, so that it returns a list of strings:

In [94]:
def get_cuisines(tr_tag):
    cuisine_tag = tr_tag.find_all('td')[4]
    if cuisine_tag.string:
        return cuisine_tag.string.split(',')
    else:
        return []

(I checked to see if the `.string` attribute is actually present, because in some cases the given `<td>` is empty.) Here's the revised loop making use of this function:

In [95]:
search_table = soup.find('table', {'class': 'search-results'}) # the search-results table
table_body = search_table.find('tbody') # get only the table body
for tr_tag in table_body.find_all('tr'):
    name = get_name(tr_tag)
    price = get_price(tr_tag)
    cuisines = get_cuisines(tr_tag)
    print(name, "-", price, "-", cuisines)

Milano Market - 2 - ['deli', ' sandwiches']
Massawa - 0 - ['ethiopian', ' african']
China Place - 0 - ['chinese', ' japanese', ' sushi']
Subsconscious - 0 - ['cheese-steaks', ' deli', ' sandwiches', ' salads']
Famous Famiglia - 0 - ['italian', ' pizza']
Kitchenette - 0 - ['bakery-pastries', ' american', ' desserts']
V & T Pizza - 0 - ['italian', ' pizza']
New Aroma - 1 - ['chinese']
Peking Garden - 1 - ['chinese']
Tom's Restaurant - 1 - ['diner']
Pisticci - 1 - ['italian']
Deluxe - 1 - ['diner', ' american']
Toast - 2 - ['american', ' bar-food']
Tom's Delicious Pizza - 2 - ['italian', ' pizza']
West Place - 1 - ['chinese']
Che' Bella Pizza - 0 - ['italian', ' pizza']
Ajanta - 0 - ['indian']
Panino Sportivo Roma - 2 - ['italian', ' coffee-tea', ' sandwiches']
Max Soha - 2 - ['italian']
Strokos Pizza - 3 - ['deli', ' pizza', ' sandwiches', ' chicken']
Camille's - 1 - ['american']
Amsterdam Restaurant - 1 - ['american-new', ' tapas']
Nussbaum & Wu - 0 - ['deli', ' sandwiches', ' bagels', 

Nice! Now we're going to rework the code so that instead of just printing stuff out, we'll make a list of dictionaries containing the data.

In [98]:
restaurants = []
search_table = soup.find('table', {'class': 'search-results'}) # the search-results table
table_body = search_table.find('tbody') # get only the table body
for tr_tag in table_body.find_all('tr'):
    name = get_name(tr_tag)
    price = get_price(tr_tag)
    cuisines = get_cuisines(tr_tag)
    rest_dict = {'name': name, 'price': price, 'cuisines': cuisines}
    restaurants.append(rest_dict)
restaurants

[{'cuisines': ['deli', ' sandwiches'], 'name': 'Milano Market', 'price': 2},
 {'cuisines': ['ethiopian', ' african'], 'name': 'Massawa', 'price': 0},
 {'cuisines': ['chinese', ' japanese', ' sushi'],
  'name': 'China Place',
  'price': 0},
 {'cuisines': ['cheese-steaks', ' deli', ' sandwiches', ' salads'],
  'name': 'Subsconscious',
  'price': 0},
 {'cuisines': ['italian', ' pizza'], 'name': 'Famous Famiglia', 'price': 0},
 {'cuisines': ['bakery-pastries', ' american', ' desserts'],
  'name': 'Kitchenette',
  'price': 0},
 {'cuisines': ['italian', ' pizza'], 'name': 'V & T Pizza', 'price': 0},
 {'cuisines': ['chinese'], 'name': 'New Aroma', 'price': 1},
 {'cuisines': ['chinese'], 'name': 'Peking Garden', 'price': 1},
 {'cuisines': ['diner'], 'name': "Tom's Restaurant", 'price': 1},
 {'cuisines': ['italian'], 'name': 'Pisticci', 'price': 1},
 {'cuisines': ['diner', ' american'], 'name': 'Deluxe', 'price': 1},
 {'cuisines': ['american', ' bar-food'], 'name': 'Toast', 'price': 2},
 {'cuis

Now `restaurants` is a *list of dictionaries*, which is the data structure we were aiming for from the beginning.

## Back to SQL

In order to store this data in SQL, we first need to create the data schema. By this I mean: we need to decide which tables are needed, and what the data types will be for those tables.

So what tables do we need? Clearly we need a table for restaurants, since that's what the data represents. The restaurant table will need at least a field for name and price. The question of how to store the cuisine types is a little bit different: it's an example of a many-to-many relationship. Each restaurant can have zero or more cuisine types, and each cuisine type can be associated with more than one restaurant.

The best way to represent a many-to-many relationship in SQL is with a linking table, and this instance will be no different! Because the cuisine types themselves don't have any extra data associated with them, we can safely store a string for the type in the linking table itself, instead of creating a second table for cuisine types.

Here's what I propose for the schemas of the two tables:

The `restaurant` table:

| column | type |
| ------ | ---- |
| id | serial |
| name | varchar(80) |
| price | int |

The `cuisine` table:

| column | type |
| ------ | ---- |
| restaurant_id | int |
| cuisine | varchar(40) |

The `serial` type is a special PostgreSQL type that automatically creates an incrementing unique integer in that field. This is great for fields that function as unique identifiers where you don't want to keep track of the number yourself.

### Creating the tables

At this point, you can open up `psql` and create a new database to contain the restaurant data. We'll call it `menupages`:

    allison=# create database menupages;
    CREATE DATABASE

Connect to the database like so:

    \c menupages
    
We'll use the [`CREATE TABLE`](https://www.postgresql.org/docs/current/static/sql-createtable.html) SQL command to create the tables. Here's the `CREATE TABLE` for restaurants:

    CREATE TABLE restaurant (
      id serial,
      name varchar(80),
      price int
    );
    
And the `CREATE TABLE` for cuisines:

    CREATE TABLE cuisine (
      restaurant_id int,
      cuisine varchar(40)
    );
    
After executing these `CREATE TABLE` statements, running `\d` should show something like this:

```
menupages=# \d
                List of relations
 Schema |       Name        |   Type   |  Owner  
--------+-------------------+----------+---------
 public | cuisine           | table    | allison
 public | restaurant        | table    | allison
 public | restaurant_id_seq | sequence | allison
(3 rows)
```

(The `restaurant_id_seq` entry is something that PostgreSQL adds automatically to keep track of the `serial` type.)

### Inserting data into SQL

Now we can take the data in our list of dictionaries and add it to SQL. We'll do this by connecting to the database with pg8000 and using the `INSERT` statement to add records to the tables. The code in the following cell connects to PostgreSQL:

In [76]:
import pg8000
conn = pg8000.connect(database="menupages")

The process of inserting a record into the database from Python is a little bit complicated. First, we need to create a cursor:

In [77]:
cursor = conn.cursor()

Then, we'll write a string that contains the `INSERT INTO` SQL statement. The `INSERT INTO` statement looks like this, schematically (fill in the lower-case 

    INSERT INTO table_name (col1, col2, col3, col4, ...)
    VALUES (value1, value2, value3, val4, ...)
    RETURNING rval1, rval2, ...
    
... where:

* `table_name` is the name of your table
* `colN`: one or more comma-separated column names
* `valueN`: one or more comma-separated values to insert into the columns (in same order as `colN`)
* `rvalN`: values to *return* from the insert process

The following statement inserts a new row into the `restaurant` table with the name `Test Restaurant` and price `3`, returning the `id` field:

In [78]:
test_insert = "INSERT INTO restaurant (name, price) VALUES ('Test Restaurant', 3) RETURNING id"

We'll execute this statement with `cursor.execute()`:

In [79]:
cursor.execute(test_insert)

Using `cursor.fetchone()`, we can get back the value for the `id` field, which was automatically created by the database because the column's type is `serial`:

In [80]:
rowid = cursor.fetchone()[0] # `fetchone()` always returns a list of values, even if there's only one column
print(rowid)

267


The value for `id` will start with 0 and increment once for every record you add to the table. Note that the number keeps growing *even if you've deleted rows from the table* (which is why the number you see above is greater than 0: I've been messing with this table a lot, making mistakes, deleting records, etc.).

Changes you make to the database don't take effect until you've executed the `.commit()` method of the connection object.

In [81]:
conn.commit()

We'll do a quick check to ensure that the record did indeed get added to the table:

In [84]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM restaurant")
for item in cursor.fetchall():
    print(item)

[267, 'Test Restaurant', 3]


Looks good! We got back the row that we just created.

### Parameters

If you're including data from another source in a SQL statement, you should use *parameters* instead of attempting to construct the query yourself. This saves you from accidentally introducing errors into your program if the data you want to insert uses special SQL characters. To use parameters, we'll replace the hard-coded data items in the SQL statement we wrote above with the special replacement character sequence `%s`:

In [85]:
insert_statement = "INSERT INTO restaurant (name, price) VALUES (%s, %s) RETURNING id"

Then, when calling `.execute()` on the cursor object, we include the values we want to be included in the query as a second parameter, in a list, in the order we want them to replace the `%s` sequences in the query:

In [86]:
cursor = conn.cursor()
cursor.execute(insert_statement, ["Another Test Restaurant", 4])
rowid = cursor.fetchone()[0]
conn.commit()
print(rowid)

268


More notes TK, but here's the loop that adds all of the restaurants to the database:

In [99]:
cursor = conn.cursor()
for item in restaurants:
    cursor.execute(
        "INSERT INTO restaurant (name, price) VALUES (%s, %s) RETURNING id",
        [item['name'], item['price']])
conn.commit()